## <span style="background:red;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Work In progress</span>

* **This is the summary of Learning through compitition Google ventilator pressure prediction**

* **LSTM and RNN are not good for tabular data, this are very different flavour Neural network, although they outperforms LGBM,XGBOOST and feed forward NN in time series data**

* **LGBM model** [Google brain GCVPP](https://www.kaggle.com/shivansh002/lgbm-lover-s)

* **LSTM model** [Google brain GCVPP](https://www.kaggle.com/shivansh002/i-am-groot-tpu-war)



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt
import tensorflow as tf
from tensorflow import keras
from warnings import filterwarnings
filterwarnings('ignore')


## <span style="background:red;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Recurrent Neural Networks</span>

* **In Our beloved feed forward neural network information only flows in single direction**
* **traditional neural network is not good for sequential data**
* **data like time series and text.**
* **decision at a node(single neural cell) is made only on current input, no data of past and no future scope**

****
![fnn](https://th.bing.com/th/id/OIP.W_Ff5HIQ15akx48m1PAe7AAAAA?pid=ImgDet&rs=1)

****

* **A recurrent neural network is a traditional neural network except that a memory-state is added to the neurons.**
* **the output from node in not only sent to next layer and also to itself**


![df](https://cdn.guru99.com/images/tensorflow/082918_1006_RNNRecurren1.png)

****

* **this gif makes concept clear the input "NYU NLP Rocks !" its for single RNN cell**


![rnn](https://i.ibb.co/LdL7CM5/rnn-move-rnn-cell.gif)


## <span style="background:Red;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Data preparation</span>

In [ ]:
from sklearn.preprocessing import StandardScaler

def create_row_features(df):
    df['row_sum'] = df.sum(axis=1)
    df['row_mean'] = df.mean(axis=1)
    df['row_std'] = df.std(axis=1)
    
    return df



train1 = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

train=create_row_features(train1)
test=create_row_features(test)

X = train.drop(columns=['id','target'])
y = train['target']

test = test.drop('id',axis=1)

scaler = StandardScaler()

X=scaler.fit_transform(X)
X_test=scaler.transform(test)


## <span style="background:RED;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">LSTM </span>

* **RNN have their own problems like vanishing gradients and exploding gradient**
* **here is monstrous looking LSTM architecture but i'll try to explain in more philosophical way ;)**


![lstm](https://miro.medium.com/max/700/1*qToyitOZkf7Nhvr1LwxWgQ.png)

****
# part1
* **Forget gate**
* **identifies only the relevent information from previous state**
* **previous state: "shyam likes ice-cream but raju like bread"**
* **current state: "raju will meet alia in evening"**
* **in LSTM cell the info about raju is relevent because its being used in current state while the info of shyam is irrelevent.** 
* **the input from previous state is compared with current state, so that may be past data is significant for future data**

# part2
* **input gate**
* **next part decide what new information to store in the cell state how much of past information should be added to current state**

# part3
* **output gate**
* **Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version**

In [ ]:
#train=np.array(X)
train = X.reshape(-1, 60, X.shape[-1])
X_test=X_test.reshape(-1, 60, X_test.shape[-1])
y=y.to_numpy().reshape(-1,60)
train.shape,X_test.shape

**LSTM expects inputs to be 3 Dimensional**

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers



def Lstm_model():
    model = Sequential([
                layers.Input(shape = train.shape[-2:]),
                layers.Bidirectional(layers.LSTM(98, return_sequences=True)),
                layers.Dense(156, activation='selu'),
                layers.Bidirectional(layers.LSTM(46, return_sequences=True)),
                layers.Dense(64, activation='selu'),
                layers.Dense(1,activation='sigmoid')])
    
    return model

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers


cv = KFold(n_splits=5, shuffle=True, random_state=1)
i =0
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(train, y)):
    X_train, y_train = train[idx_train], y[idx_train]
    X_valid, y_valid = train[idx_valid], y[idx_valid]
    
    print(X_train.shape,y_train.shape)
    model = Lstm_model()
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['AUC',
                  tf.keras.metrics.BinaryAccuracy(),
                  tf.keras.metrics.FalseNegatives()])

    
    print(f"training for Fold {i+1}")
    
    LSTM = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=512,
        epochs=70)
                 
                 
    prediction = model.predict(X_test)
    predictions.append(prediction)

In [ ]:
predict=[]
for l in range(5):
    flat_ls = [item for sublist in predictions[l] for item in sublist]
    predict.append(flat_ls)   

sub=np.zeros(540000)
print(sub.shape)
for pred in predict:
    pred=np.array(pred).reshape(540000)
    sub+=pred

    
sub=sub/5

submission['target'] = sub
submission.to_csv('lstm.csv', index=False)

# LGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
test = test.drop('id',axis=1)

In [ ]:
X = train1.drop(columns=['id','target'])
y = train1['target']

skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

pred=[]


for fold,(idx1,idx2) in enumerate(skf.split(X,y)):
    
    Xtrain,ytrain=X.iloc[idx1],y.iloc[idx1]
    Xvalid,yvalid=X.iloc[idx2],y.iloc[idx2]
    
    model=LGBMClassifier(n_estimators=1500,learning_rate=0.02,device='gpu',num_leaves=900,
                         min_child_samples=60)
    
    model.fit(Xtrain,ytrain,eval_set=[(Xvalid,yvalid)],early_stopping_rounds=100,verbose=50)
    
    valid_pred = model.predict_proba(Xvalid)[:,1]
    test_pred = model.predict_proba(test)[:,1]
    
    pred.append(test_pred)
    
    score = roc_auc_score(yvalid, valid_pred)
    print(score)
    